Run all the cells and then enter your case in

In [ ]:
# Install all necessary dependencies
!pip install dotenv
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 842.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# import all libraries used
from dotenv import load_dotenv
from __future__ import annotations
import os
import openai
from openai import OpenAI
from typing import List, Dict
import pandas as pd
from tqdm import tqdm

In [ ]:
# Sets the environment variable (access token) -- tried to but it didn't work
load_dotenv()

True

In [ ]:
# Create a variable to record conversation
global_history: List[Dict[str,str]]= []

In [ ]:
# Class which defines basic functionality of an entity
class Person:

    # Constructor sets all the variables and connects with hugging-face
    def __init__(self, name: str, system_prompt: str, model: str):
        self.name= name
        self.system_prompt= system_prompt.strip()
        self.modell= model
        # self.client= InferenceClient(model, token= os.getenv("ACCESS_TOKEN"))
        self.client = OpenAI(
            # api_key="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",
            api_key="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",
            base_url="https://api.groq.com/openai/v1"
        )

    # Helper Function to format the prompt before passing as input
    def _format_prompt(self, msg: str)-> str:

        # Gather all history of conversation in a single list of dictionary, along with system prompt and current prompt
        messagess= [{"role": "system", "content": self.system_prompt}]
        global_history[-2:] # Only give previous 2 dialogues as context
        messagess.extend(global_history)
        messagess.append({"role": "user", "content": msg})
        return messagess

    # Function to produce output from the model by passing prompt
    def respond(self, msg: str, should_record: bool, **gen_kwargs)-> str:

        # Format the input and pass it to model to get output
        prompt= self._format_prompt(msg)
        # Generate Response
        response = self.client.chat.completions.create(
        model=self.modell,
        messages=prompt,
        temperature=0.7,
        max_tokens=512
        )
        answer=(response.choices[0].message.content).strip()

        # Append the latest response to history
        if should_record == True:
            global_history.append({"role": "user", "content": msg})
            global_history.append({"role": "assistant", "content": answer})
        return answer

In [ ]:
# Define variables for each entity of court room
modelll = "llama-3.3-70b-versatile"
personality_judge= "You are role-playing a judge and currently judging this case. Don't give any follow up or summary."
personality_defending_lawyer= "You are role-playing a defending the defendant of this case. Don't give any follow up or summary."
personality_prosecuting_lawyer= "You are role-playing a prosecuting lawyer of the plaintiff of this case. Don't give any follow up or summary."
personality_defendant= "You are role-playing a defendant of this case. Don't give any follow up or summary."
personality_plaintiff= "You are role-playing a plaintiff of this case. Don't give any follow up or summary."
personality_witness= "You a role-playing a witness of this case. Don't give any follow up or summary."

In [ ]:
# Define all entities of court room and a regulating system that regulates the conversation
judge= Person("Judge", personality_judge, modelll)
defending_lawyer= Person("Denfending Lawyer", personality_defending_lawyer, modelll)
prosecuting_lawyer= Person("Prosecuting Lawyer", personality_prosecuting_lawyer, modelll)
defendant= Person("Defendant", personality_defendant, modelll)
plaintiff= Person("Plaintiff", personality_plaintiff, modelll)

In [ ]:
# Define in-simulation prompts
judge_prompt= "Give your inference of conversation. If you think the case is solved based on previous conversation, say the case is over and write _OVER_ at the end of the response, then tell the verdict of whether the case is granted or denied. If the case is granted write _GRANTED_ in your response or if the case is denied, write _DENIED_ in your response. Else ask the prosecution lawyer to present his next argument."
prosecuting_lawyer_prompt= "Proceed with your next argument or question and try to win the case. If you want to call a witness, write _WITNESS_ at the end of response and specify his or her personality, if you want answer from defendant, write _DEFENDANT_ at the end of response or if you want answer from plaintiff, write _PLAINTIFF_ at the end of your response. "
defending_lawyer_prompt= "Defend your client by countering the argument of prosecution lawyer and try to win the case."
plaintiff_prompt= "Answer in response to the question from prosecuting lawyer and try to win the case."
defendant_prompt= "Answer in response to the question from prosecuting lawyer and try to win the case."
witness_prompt= "You are a witness in this case. Take personality as said by Prosecuting lawyer. Answer in response to the question from prosecuting lawyer and state truly whatever is asked."

In [ ]:
# case= input("Enter the case: ")
def generate_convANDverdict(case):
    verdict=1
    turn= 1
    MAX_TURNS= 5  # Max rounds to be simulated

    #Initial statement giving intro to the case
    out= f"""\nJudge: {judge.respond("Give your permission to proceed the case by stating it and saying proceed. The case is "+case, True)}\n"""
    print(out)
    out= f"""\nProsecuting Lawyer: {prosecuting_lawyer.respond("After reading previous conversation, present your side of the case, and present in a way to win the case", True)}\n"""
    print(out)
    out= f"""\nDefending Lawyer: {defending_lawyer.respond("After reading previous conversation, present your side of the case.", True)}\n"""
    print(out)

    # Loop to simulate the conversation round wise
    while turn<MAX_TURNS:
        print(f"\n===== TURN {turn} =====\n")
        out= f"""\nJudge: {judge.respond(judge_prompt, True)}\n"""
        print(out)
        if "_OVER_" in out:
          if "_GRANTED_" in out:
            verdict=1
            break
          elif "_DENIED_" in out:
            verdict=0
            break
        else:
          out= f"""\nProsecuting Lawyer: {prosecuting_lawyer.respond(prosecuting_lawyer_prompt, True)}\n"""
          print(out)
          if "_DEFENDANT_" in out:
            out= f"""\nDefendant: {defendant.respond(defendant_prompt, True)}\n"""
            print(out)
          elif "_PLAINTIFF_" in out:
            out= f"""\nPlaintiff: {plaintiff.respond(plaintiff_prompt, True)}\n"""
            print(out)
          elif "_WITNESS_" in out:
            witness= Person("Witness", personality_witness, modelll)
            out= f"""\nWitness: {witness.respond(witness_prompt, True)}\n"""
            print(out)
            del witness
        out= f"""\nDefending Lawyer: {defending_lawyer.respond(defending_lawyer_prompt, True)}\n"""
        print(out)
        if "_WITNESS_" in out:
            witness= Person("Witness", personality_witness, modelll)
            out= f"""\nWitness: {witness.respond(witness_prompt, True)}\n"""
            print(out)
            del witness
        turn+= 1
    if(turn==(MAX_TURNS+1)):
      out= f"""\nJudge: {judge.respond("Give your decision of giving another date for the case's hearing by stating it and saying postponed. The case is "+case, True)}\n"""
      print(out)
      return global_history, verdict

In [ ]:
# Number of rows in dataset to be considered
rows = 50
# Variables that stores verdict and conversation
judge_verdict = 0
answers=[]
# Create a file to store final output
df = pd.DataFrame(columns=["id", "verdict"])
# Read the csv file
data= pd.read_csv("cases.csv")

In [ ]:
judgee= Person("Judge","You are role-playing a judge and currently judging this case. Don't give any follow up or summary. Judge the case based on case description. tell the verdict of whether the case is granted or denied. If the case is granted write _GRANTED_ in your response or if the case is denied, write _DENIED_ in your response","llama-3.3-70b-versatile")

In [ ]:
for j in tqdm(range(rows)):
  try:
    result = judgee.respond(data['text'][j][:4000].strip(), False)
    # answers[j],judge_verdict= result
    # print(answers[j])
    if "_DENIED_" in answer:
            judge_verdict = 0
    else:
            judge_verdict = 1
    df.loc[len(df)] = [data['id'][j], judge_verdict]
  except Exception as e:
    print(e)
    print(f"Skipping row {j}")
    judge_verdict=1
    df.loc[len(df)] = [data['id'][j], judge_verdict]

  2%|▏         | 1/50 [00:00<00:10,  4.73it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98733, Requested 6478. Please try again in 1h15m1.741999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 0
Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98733, Requested 6478. Please try again in 1h15m1.536s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 1


  8%|▊         | 4/50 [00:00<00:09,  4.87it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98732, Requested 6478. Please try again in 1h15m1.306s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 2
Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98732, Requested 6478. Please try again in 1h15m1.119s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 3


 10%|█         | 5/50 [00:01<00:09,  4.80it/s]

...

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98722, Requested 6478. Please try again in 1h14m52.241s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 45
Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98722, Requested 6478. Please try again in 1h14m52.041999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 46


 98%|█████████▊| 49/50 [00:10<00:00,  4.83it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98721, Requested 6361. Please try again in 1h13m10.751999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 47
Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98721, Requested 6478. Please try again in 1h14m51.639999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 48


100%|██████████| 50/50 [00:10<00:00,  4.76it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7qyg6tf67tac32yf0m4zwp` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98721, Requested 6478. Please try again in 1h14m51.446s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Skipping row 49


In [ ]:
# See the description of generated result
df.describe()

,verdict
count,50.000000
mean,0.660000
std,0.478518
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
# Convert to csv for submission
df.to_csv('output.csv', index=False)